In [27]:
import csv   

In [28]:
def downloadAllData(reader):
    L = []
    for row in reader:
        L.append(row)
    return L

In [29]:
def getList(data, indexColumn):
    L = []
    for i in range(1, len(data)):
        L.append(data[i][indexColumn])
    return L

In [30]:
def predictMin(datalist, startIndex, windowsSize):
    minSet = []
    for j in range(startIndex, startIndex + windowsSize):   
        minSet.append(float(datalist[j]))
    result = sorted(minSet)
    return result

In [31]:
def getPpercent(percent, list):
    total = len(list)
    index = float(total*percent)
    if float(index) - float(int(index)) == 0:
        index = int(index)
        
        return float((list[index]+list[index-1])/2)
    index = int(index)
    return float(list[index-1]) 

In [32]:
def predictValue(list, windowSize, percent):
    result = []
    for i in range(0,len(list)-windowSize+1):
        tempList = predictMin(list, i, windowSize)
        pValue = getPpercent(percent, tempList)
        result.append(pValue)
    return result

In [33]:
def getTrueMin(data, startblock):
    result=[]
    # start point
    for i in range(startblock, len(data)):
        result.append(float(data[i][1]))
    return result

In [34]:
def computeOnChainPercentFixRange(predictList, startIndex, endIndex, trueDataList):
    possiblityOnchain = 0
    integerRange = endIndex - startIndex
    for i in range(startIndex, endIndex):
        if float(predictList[i]) >= float(trueDataList[i]):
            possiblityOnchain +=1
    return float(possiblityOnchain / integerRange)

In [35]:
def getBestPValue( expectedPValue, originPValue, origindataList, numbertraindata, trueList ):
    error = 0.01
    startUpPValue = originPValue
    startlowPValue = 0.01
    maxRetry = 15
    result = (startlowPValue+startUpPValue)/2
    
    for i in range(maxRetry):     
        dataList = predictValue(origindataList, numbertraindata, result)      
        tempPValue =  computeOnChainPercentFixRange(dataList, 0,  numbertraindata, trueList)
        i +=1   
        if tempPValue > expectedPValue + error:
            startUpPValue = result
            result = (startUpPValue+startlowPValue)/2
        elif tempPValue < expectedPValue - error:
            startlowPValue = result
            result = (startUpPValue+startlowPValue)/2
        else:
            return result
    return originPValue

In [36]:
# The predicting data of GP 
csvFile = open("/Users/chihyunchuang/Downloads/new1-136312.csv")
reader2 = csv.reader(csvFile)
datGP = downloadAllData(reader2)
datGP = datGP[1:]

In [37]:
# Algorithm 1
def getPredictList(data,  numbertraindata, upperBoundTrainData, pValue, error):
    upperBound = pValue + error
    lowBound = pValue -error
    
    minList = []
    for i in range(1, len(data)):
        minList.append( data[i][1] )
    
    predictData = predictValue(minList,  numbertraindata, pValue)
    # the last index is unuse
    predictData = predictData[upperBoundTrainData - numbertraindata:len(predictData)-1]  
    trueData = getTrueMin(data, upperBoundTrainData+1)
     
    result = []
    # initial data
    for i in range(0, numbertraindata):
        result.append(trueData[i])
     
    for i in range(0, len(datGP) - numbertraindata):
        # Compute R value to monitor the success ratio
        nowOnChainRatio = computeOnChainPercentFixRange(predictData, i, i+numbertraindata, trueData)
        nowArray = trueData[i:i+numbertraindata]
        
        if nowOnChainRatio >= upperBound:
            # use decay method
            preductValue = predictValue(nowArray,  numbertraindata, pValue)[0]
            trainList = minList[i: i+numbertraindata+numbertraindata-1]
            trueList = minList[i+numbertraindata:i+numbertraindata+numbertraindata ]
            bestP = getBestPValue( pValue, nowOnChainRatio, trainList, numbertraindata, trueList )
            preductValue = predictValue(nowArray,  numbertraindata, pValue)[0]
            result.append(preductValue)
        elif nowOnChainRatio <= lowBound:
            # use GP Method
            GPValue = float(datGP[i+numbertraindata][1])+1.645*float(datGP[i+numbertraindata][2])      
            GSValue = predictValue(nowArray,  numbertraindata, pValue)[0]
            if GPValue < GSValue:
                GPValue = GSValue       
            result.append(preductValue)
        else:
            preductValue = predictValue(nowArray,  numbertraindata, pValue)[0]
            result.append(preductValue)
    return result

In [38]:
csvFile = open("/Users/chihyunchuang/Downloads/new_block_p2.5.csv")
reader = csv.reader(csvFile)
data = downloadAllData(reader)

In [39]:
# parameters: number of training data of GS, max{number of training data of GP, number of traing data of GS}, Pvalue, error  
expected = getPredictList(data,  30, 200, 0.95, 0.1)
trueData = getTrueMin(data, 201)

In [91]:
with open("/Users/chihyunchuang/Downloads/newDataForNewBlock1-13612.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['block_number','true_min', 'Amis-75' ])

    for i in range(1,len(datGP)-1):
        writer.writerow([str(data[i+201][0]), trueData[i], expected[i]  ])